In [1]:
#(a) (5 points) Using the pandas library, read the csv data file from your S3 bucket and
#create a data-frame called weather. Select data up to October 31, 2011. After that,
#remove any observation with missing values.
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np 

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, accuracy_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,  GradientBoostingClassifier 
from sklearn.tree import DecisionTreeClassifier

s3 = boto3.resource('s3')
bucket_name = "rachaeld-data445"
bucket = s3.Bucket(bucket_name)

file_key = 'weather.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the data-file
weather = pd.read_csv(file_content_stream)
weather.head()

,rowID,hpwren_timestamp,air_pressure,air_temp,avg_wind_direction,avg_wind_speed,max_wind_direction,max_wind_speed,min_wind_direction,min_wind_speed,rain_accumulation,rain_duration,relative_humidity
0,0,2011-09-10 00:00:49,912.3,64.76,97.0,1.2,106.0,1.6,85.0,1.0,NaN,NaN,60.5
1,1,2011-09-10 00:01:49,912.3,63.86,161.0,0.8,215.0,1.5,43.0,0.2,0.0,0.0,39.9
2,2,2011-09-10 00:02:49,912.3,64.22,77.0,0.7,143.0,1.2,324.0,0.3,0.0,0.0,43.0
3,3,2011-09-10 00:03:49,912.3,64.40,89.0,1.2,112.0,1.6,12.0,0.7,0.0,0.0,49.5
4,4,2011-09-10 00:04:49,912.3,64.40,185.0,0.4,260.0,1.0,100.0,0.1,0.0,0.0,58.8


In [4]:
#(8 points) After consulting with a meteorologist, he recommends to use the following variables for clustering purposes: air pressure,
#    air temp, avg wind direction,avg wind speed, max wind direction, max wind speed, and relative humidity. Transform all the variables
#    of interest to 0-1 scale.

## scaling 
scaler = MinMaxScaler()
weather[['air_pressure_0_1', 'air_temp_0_1', 'avg_wind_direction_0_1', 'avg_wind_speed_0_1', 'max_wind_direction_0_1',
        'max_wind_speed_0_1', 'relative_humidity_0_1']] = scaler.fit_transform(weather[['air_pressure', 'air_temp', 'avg_wind_direction','avg_wind_speed',
                                                                                      'max_wind_direction', 'max_wind_speed', 'relative_humidity']])
weather.head(10)

,rowID,hpwren_timestamp,air_pressure,air_temp,avg_wind_direction,avg_wind_speed,max_wind_direction,max_wind_speed,min_wind_direction,min_wind_speed,rain_accumulation,rain_duration,relative_humidity,air_pressure_0_1,air_temp_0_1,avg_wind_direction_0_1,avg_wind_speed_0_1,max_wind_direction_0_1,max_wind_speed_0_1,relative_humidity_0_1
0,0,2011-09-10 00:00:49,912.3,64.76,97.0,1.2,106.0,1.6,85.0,1.0,NaN,NaN,60.5,0.297959,0.488064,0.270195,0.037152,0.295265,0.041783,0.647887
1,1,2011-09-10 00:01:49,912.3,63.86,161.0,0.8,215.0,1.5,43.0,0.2,0.0,0.0,39.9,0.297959,0.474801,0.448468,0.024768,0.598886,0.038997,0.424702
2,2,2011-09-10 00:02:49,912.3,64.22,77.0,0.7,143.0,1.2,324.0,0.3,0.0,0.0,43.0,0.297959,0.480106,0.214485,0.021672,0.398329,0.030641,0.458288
3,3,2011-09-10 00:03:49,912.3,64.40,89.0,1.2,112.0,1.6,12.0,0.7,0.0,0.0,49.5,0.297959,0.482759,0.247911,0.037152,0.311978,0.041783,0.528711
4,4,2011-09-10 00:04:49,912.3,64.40,185.0,0.4,260.0,1.0,100.0,0.1,0.0,0.0,58.8,0.297959,0.482759,0.515320,0.012384,0.724234,0.025070,0.629469
5,5,2011-09-10 00:05:49,912.3,63.50,76.0,2.5,92.0,3.0,61.0,2.0,0.0,0.0,62.6,0.297959,0.469496,0.211699,0.077399,0.256267,0.080780,0.670639
6,6,2011-09-10 00:06:49,912.3,62.78,79.0,2.4,89.0,2.7,62.0,2.0,0.0,0.0,65.6,0.297959,0.458886,0.220056,0.074303,0.247911,0.072423,0.703142
7,7,2011-09-10 00:07:49,912.3,62.42,86.0,2.0,92.0,2.4,75.0,1.8,0.0,0.0,65.2,0.297959,0.453581,0.239554,0.061920,0.256267,0.064067,0.698808
8,8,2011-09-10 00:08:49,912.3,62.24,105.0,1.4,125.0,1.9,82.0,1.0,0.0,0.0,65.8,0.297959,0.450928,0.292479,0.043344,0.348189,0.050139,0.705309
9,9,2011-09-10 00:09:49,912.3,62.24,93.0,0.4,126.0,0.7,14.0,0.2,0.0,0.0,58.6,0.297959,0.450928,0.259053,0.012384,0.350975,0.016713,0.627302


In [5]:
#(8 points) Using the silhouette score, estimate the number of clusters for this dataset.
#Consider 2 to 20 clusters. Make sure to use n init = 20 in the KMeans function from
#the sklearn.cluster library.
from sklearn.cluster import KMeans
from skfuzzy.cluster import cmeans
from sklearn.metrics import silhouette_score

## defining a lists to store silhouette scores
silhouette = list()

for i in range (2,20):
    ##clustering the data into i clusters
    kmeans_md = KMeans(n_clusters = i, n_init = 20).fit(weather[['air_pressure_0_1', 'air_temp_0_1', 'avg_wind_direction_0_1', 'avg_wind_speed_0_1', 'max_wind_direction_0_1',
        'max_wind_speed_0_1', 'relative_humidity_0_1']])
    kmeans_md_labels = kmeans_md.labels_
    
    ##computing the silhouette score
    score = silhouette_score(weather[['air_pressure_0_1', 'air_temp_0_1', 'avg_wind_direction_0_1', 'avg_wind_speed_0_1', 'max_wind_direction_0_1',
        'max_wind_speed_0_1', 'relative_humidity_0_1']], kmeans_md_labels)
    silhouette.append(score)

ModuleNotFoundError: No module named 'skfuzzy'